<a href="https://colab.research.google.com/github/TwistedGrims/sdfsdf32423dsfsdzxvczxfweqr2314/blob/main/imagegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Imports + google drive
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -U xformers
from IPython.display import display
import torch
import random
from diffusers import (
    StableDiffusionXLPipeline,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
)
from PIL import Image
from IPython.display import display
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

clear_output(wait=True)


In [ ]:
#@markdown # load checkpoints


checkpoint = "nova3dcgxl" #@param["nova3dcgxl","nsfwillustrious","pornmaster"]
checkpoint_paths = {
    "pornmaster": "/content/gdrive/MyDrive/Pmaster.safetensors",
    "nsfwillustrious": "/content/gdrive/MyDrive/NSFWillustrious.safetensors",
    "nova3dcgxl": "/content/gdrive/MyDrive/nova3DCGXL_illustriousV60.safetensors"
}

model_path = checkpoint_paths[checkpoint]
pipe = StableDiffusionXLPipeline.from_single_file(
    model_path,
    torch_dtype=torch.float16
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_slicing()
#pipe.enable_sequential_cpu_offload()
clear_output(wait=True)


In [ ]:

import time
prompt = "masterpiece, best quality, amazing quality, explicit, from side, profile, 2girls, solo, turquoise hair, high ponytail, heterochromia, red eyes, blue eyes, glowing eyes, eyebrows, eyelashes, head tilt, nipples, latex dress, side slit, outdoors, standing on balcony, overlooking futuristic city, neon lights reflecting, streets below, holographic billboards, glowing advertisements, night cityscape, cyberpunk atmosphere, soft ambient glow, intricate details, blurry background"  #@param {type:"string"}
negative_prompt = "child, loli, patreon logo, bad quality, worst quality, worst detail, sketch, censor, censored, watermark, signature, blurry, plain simple background, blank eyes, messy hair, hydrokinesis, earrings, facial tattoo,"  #@param {type:"string"}
firsttime =time.time()
aspect_ratios = {
    "Square": (512,512),
    "Landscape": (1216, 832),
    "Portrait": (832, 1216)
}
chosen_ratio = "Square"  #@param ["Square", "Landscape", "Portrait"]
width, height = aspect_ratios[chosen_ratio]
cfg_scale = 4  #@param {type:"number"}
steps = 20  #@param {type:"integer"}
seed = 0  #@param {type:"integer"}

if seed and seed > 0:
    current_seed = seed
else:
    current_seed = random.randint(0, 2**32 - 1)

generator = torch.Generator(device="cuda").manual_seed(current_seed)

def callback(step: int, timestep: int, latents):
    with torch.no_grad():
        image = pipe.vae.decode(latents / pipe.vae.config.scaling_factor).sample
        image = (image / 2 + 0.5).clamp(0, 1)
        image = (image * 255).permute(0, 2, 3, 1).cpu().numpy().astype("uint8")[0]
        pil_img = Image.fromarray(image)
        preview = pil_img.copy()
        preview.thumbnail((width//2, height//2), Image.LANCZOS)
        clear_output(wait=True)
        display(preview)
        print(f"Step {step+1}/{steps}")

result = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=steps,
    guidance_scale=cfg_scale,
    generator=generator,
    callback=callback,
    callback_steps=1
)

clear_output(wait=True)
display(result.images[0])

torch.cuda.empty_cache()
print(f"took {(time.time()-firsttime):.2f} seconds")